# BUILD THE NEURAL NETWORK
  
Neural networks comprise of layers/modules that perform operations on data.   
comprise of : ~로 구성되는  
신경망은 데이터에 대한 연산을 수행하는 층/모듈로 구성됨  
  
The torch.nn namespace provides all the building blocks you need to build your own neural network.   

Every module in PyTorch subclasses the nn.Module.  
PyTorch의 각 모듈은 nn.Module을 서브클래스화  
하위 클래스가 상위 클래스를 상속받아 새로운 클래스를 정의하는 것  
상위 클래스는 일반적인 특성이나 기능을 정의하고,   
하위 클래스는 이러한 기능을 상속받아 추가하거나 수정하여 특정한 동작을 수행하도록 만든다  
PyTorch의 nn.Module은 신경망 모델의 기본 클래스로 사용됨  
사용자는 이 클래스를 서브클래스화하여 자신만의 신경망 아키텍처를 정의할 수 있음  
(기존의 기능을 상속받아 사용하면서 필요한 추가적인 동작을 구현)  

A neural network is a module itself that consists of other modules (layers).     
consist of : ~로 구성되다
신경망은 그 자체로 모듈 / 다른 모듈(레이어로 구성되는)  

This nested structure allows for building and managing complex architectures easily.  
    
In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.  

In [5]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training
  
We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available.   
Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.  
  
torch.cuda : 현재 시스템에서 GPU가 사용 가능한지 여부  
torch.backends.mps : Apple M1에서 GPU 사용  

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


### Define the Class
  
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.   
  
Every nn.Module subclass implements the operations on input data in the forward method.  
implement : 구현하다, 실행하다  

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # 28x28 픽셀의 이미지를 평탄화된 1D 텐서로 변환
                                   # 입력 28x28 = 784, 출력 512
            nn.ReLU(),
            nn.Linear(512, 512), # 입력 차원이 512고 출력 차원도 512
            nn.ReLU(),
            nn.Linear(512, 10), # 10개의 클래스에 대한 분류를 수행하기 때문에 출력 10
                                # 최종 출력 레이어이기 때문에 활성화 함수 없음
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.  
인스턴스 : 어떤 클래스에 속하는 각각의 객체 

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. 모델 사용을 위해 입력 데이터를 전달  
This executes the model’s forward, along with some background operations.   
execute : 실행하다  , 일부 백그라운드 작업 또한 수행  
Do not call model.forward() directly!  
  
Calling the model on the input returns   
a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class,   
and dim=1 corresponding to the individual values of each output.  
corresponding : ~에 해당하는   
dim=0은 각 클래스에 대한 10개의 원시 예측 값 출력에 해당  
dim=1은 각 출력의 개별 값에 해당하는 2차원 텐서가 반환  
  
We get the prediction probabilities by passing it through an instance of the nn.Softmax module.  

In [9]:
X = torch.rand(1, 28, 28, device=device)
# device=device : 현재 활성화된 디바이스에 텐서를 할당

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)

y_pred = pred_probab.argmax(1)
# pred_probab에서 최대값의 인덱스를 추출

print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


---

### Model Layers

Let’s break down the layers in the FashionMNIST model.   
break down : (분석할 수 있도록) 나누어지다  
각 레이어에서 어떤 일이 일어나는지 나누어서 분석하겠다는 뜻  
이외에 고장나다, 실패하다, 나빠지다, 감정을 주체하지 못하다 라는 뜻도 있음  

To illustrate it, we will take a sample minibatch of 3 images of size 28x28   
and see what happens to it as we pass it through the network.

In [10]:
input_image = torch.rand(3,28,28)
# 3은 RGB 이미지의 빨간색, 초록색, 파란색 3채널을 의미
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
  
We initialize the nn.Flatten layer to convert each 2D 28x28 image   
into a contiguous array of 784 pixel values   
(the minibatch dimension (at dim=0) is maintained).  

contiguous : 인접한, 근접한 / contiguous array : 연속 배열  
연속 배열 : 메모리 상에서 연속된 위치에 요소가 저장된 배열

각 2D 28x28 이미지를 784개의 픽셀 값으로 이루어진 연속 배열로 변환    
(dim=0에서 미니배치 차원은 유지)  

즉, nn.Flatten 레이어를 사용하여 이미지를 펼치고,  
이를 통해 연속된 1차원 배열로 변환

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.  
저장된 가중치(weights)와 편향(biases)을 사용하여 입력에 선형 변환을 적용하는 모듈  

In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
# out_features : 출력 특성(feature)의 수

hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
  
Non-linear activations are what create the complex mappings between the model’s inputs and outputs.   
They are applied after linear transformations to introduce nonlinearity,   
helping neural networks learn a wide variety of phenomena.  
phenomena : phenomenon의 복수,  phenomenon : 현상  

In this model, we use nn.ReLU between our linear layers,   
but there’s other activations to introduce non-linearity in your model.  

In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1918,  0.6809, -0.1279, -0.1658,  0.1338,  0.2457, -0.4145,  0.7269,
         -0.1419, -0.3149, -0.0689, -0.3511, -0.2054, -0.5268, -0.1730,  0.3882,
          0.0594,  0.3020,  0.4584,  0.1076],
        [-0.1275,  0.4373, -0.2131, -0.2013,  0.1494,  0.0678, -0.6561,  0.4959,
         -0.2742, -0.1730, -0.2247,  0.0493, -0.0193, -0.7585, -0.1401,  0.2591,
          0.0996,  0.1826,  0.1835,  0.1200],
        [ 0.0102,  0.9076, -0.3240, -0.1586,  0.1789,  0.2827, -0.4057,  0.4855,
          0.1354, -0.1306, -0.0149, -0.3936, -0.1112, -0.4410, -0.0464,  0.1850,
         -0.3303, -0.0129,  0.7587,  0.2497]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1918, 0.6809, 0.0000, 0.0000, 0.1338, 0.2457, 0.0000, 0.7269, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3882, 0.0594, 0.3020,
         0.4584, 0.1076],
        [0.0000, 0.4373, 0.0000, 0.0000, 0.1494, 0.0678, 0.0000, 0.4959, 0.0000,
         0.0000, 0.0000, 0.0493, 0.0000, 0.0000, 0.00

#### nn.Sequential
  
nn.Sequential is an ordered container of modules. 
순서가 있는 자료구조    
The data is passed through all the modules in the same order as defined.   
You can use sequential containers to put together a quick network like seq_modules.  
순차적으로 모듈들을 나열하여 빠르게 네트워크를 구성할 수 있음  

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10) # 입력 특성이 20이고 출력 특성이 10
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax
  
The last linear layer of the neural network returns logits - raw values in [-infty, infty] -   
which are passed to the nn.Softmax module.   
The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class.   
dim parameter indicates the dimension along which the values must sum to 1.

indicate 나타내다  
dim 매개변수는 확률 값이 1이 되어야 하는 차원을 지정  
즉, 확률값들의 합이 1이 되도록 정규화될 차원을 나타냄   
보통은 클래스 차원이며, 각 샘플에 대한 확률값들의 합이 1이 되도록 조정

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters
  
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training.   
Subclassing nn.Module automatically tracks all fields defined inside your model object  
nn.Module을 하위 클래스로 만들면 모델 객체 내에서 정의된 모든 필드를 자동으로 추적하고,  
and makes all parameters accessible using your model’s parameters() or named_parameters() methods.  
모델의 parameters() 또는 named_parameters() 메서드를 사용하여 모든 매개변수에 접근할 수 있게 됨  

In this example, we iterate over each parameter, and print its size and a preview of its values.  
이 예제에서는 각 매개변수를 반복하고 해당 크기와 값의 미리보기를 출력  

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0264,  0.0338,  0.0196,  ...,  0.0181, -0.0275,  0.0027],
        [ 0.0006,  0.0207, -0.0185,  ..., -0.0165,  0.0205,  0.0253]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0186, -0.0023], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0029,  0.0192,  0.0081,  ...,  0.0134, -0.0224, -0.0006],
        [-0.0430, -0.0121, -0.0209,  ...,  0.0297, -0.0065, -0.0327]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 